# General Imports and Pre-Requisites

In [2]:
import carla
import math
import random
import time
import numpy as np
import cv2
import copy

from carla import Transform, Location
from utils import download_installer, get_actor_blueprints, remove_actors

import settings

settings.init()

In [3]:
download_installer()

# Connect to the client and get the world object
client = carla.Client("localhost", settings.carla_port)
world = client.get_world()

# Get the blueprint library and the spawn points for the map
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

all_id = []

# Add Camera

In [3]:
cameras = []
camera_data = {}

# Callback stores sensor data in a dictionary for use outside callback
def camera_callback(image, camera, data_dict):
    # create image based on flatten array (4 stands for RGBA)
    data_dict[camera.id]["image"] = np.reshape(
        np.copy(image.raw_data), (image.height, image.width, 4)
    )

In [4]:
for i in range(settings.number_of_cameras):
    spectator_location = world.get_spectator().get_location()
    camera_bp = bp_lib.find("sensor.camera.rgb")
    camera_init_trans = Transform(carla.Location())

    camera = world.spawn_actor(
        camera_bp,
        Transform(
            Location(spectator_location.x, spectator_location.y, spectator_location.z)
        ),
    )
    cameras.append(camera)

    # Get gamera dimensions and initialise dictionary
    image_w = camera_bp.get_attribute("image_size_x").as_int()
    image_h = camera_bp.get_attribute("image_size_y").as_int()
    camera_data[camera.id] = {"image": np.zeros((image_h, image_w, 4))}

    # Start camera recording
    camera.listen(lambda image: camera_callback(image, camera, camera_data))

    if i >= settings.number_of_cameras:
        break
    else:
        input("Press Enter to continue...")

for camera in cameras:
    # OpenCV named window for rendering
    cv2.namedWindow("RGB Camera " + str(camera.id), cv2.WINDOW_AUTOSIZE)
    cv2.imshow("RGB Camera " + str(camera.id), camera_data[camera.id]["image"])
    cv2.waitKey(1)

# Game loop
while True:
    for camera in cameras:
        # Imshow renders sensor data to display
        cv2.imshow("RGB Camera " + str(camera.id), camera_data[camera.id]["image"])

    # Quit if user presses 'q'
    key = cv2.waitKey(1)
    if key == 27:
        break

# Close OpenCV window when finished
cv2.destroyAllWindows()

In [29]:
spectator_transformation = world.get_spectator().get_transform()

inst_camera_bp = bp_lib.find('sensor.camera.instance_segmentation') 
inst_camera = world.spawn_actor(inst_camera_bp, Transform(
            Location(spectator_transformation.location.x, spectator_transformation.location.y, spectator_transformation.location.z), carla.Rotation(spectator_transformation.rotation.pitch, spectator_transformation.rotation.yaw, spectator_transformation.rotation.roll )
        ))

def inst_callback(image, data_dict):
    data_dict['inst_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

    print(data_dict['inst_image'])


# Get gamera dimensions and initialise dictionary                       
image_w = inst_camera_bp.get_attribute("image_size_x").as_int()
image_h = inst_camera_bp.get_attribute("image_size_y").as_int()
camera_data = {'inst_image': np.zeros((image_h, image_w, 4))}

# Start camera recording
inst_camera.listen(lambda image: inst_callback(image, camera_data))

# OpenCV named window for rendering
cv2.namedWindow("Inst Camera " + str(inst_camera.id), cv2.WINDOW_AUTOSIZE)
cv2.imshow("Inst Camera " + str(inst_camera.id), camera_data["inst_image"])
cv2.waitKey(1)

# Game loop
while True:
    cv2.imshow("Inst Camera " + str(inst_camera.id), camera_data["inst_image"])

    # Quit if user presses 'q'
    key = cv2.waitKey(1)
    if key == 27:
        break

# Close OpenCV window when finished
cv2.destroyAllWindows()
inst_camera.stop()

[[[152 198   9 255]
  [152 198   9 255]
  [152 198   9 255]
  ...
  [116 194  13 255]
  [116 194  13 255]
  [116 194  13 255]]

 [[152 198   9 255]
  [152 198   9 255]
  [152 198   9 255]
  ...
  [116 194  13 255]
  [116 194  13 255]
  [116 194  13 255]]

 [[152 198   9 255]
  [152 198   9 255]
  [152 198   9 255]
  ...
  [116 194  13 255]
  [116 194  13 255]
  [116 194  13 255]]

 ...

 [[152 198   9 255]
  [152 198   9 255]
  [152 198   9 255]
  ...
  [154  36   8 255]
  [154  36   8 255]
  [154  36   8 255]]

 [[152 198   9 255]
  [152 198   9 255]
  [152 198   9 255]
  ...
  [154  36   8 255]
  [154  36   8 255]
  [154  36   8 255]]

 [[152 198   9 255]
  [152 198   9 255]
  [152 198   9 255]
  ...
  [154  36   8 255]
  [154  36   8 255]
  [154  36   8 255]]]
[[[152 198   9 255]
  [152 198   9 255]
  [152 198   9 255]
  ...
  [116 194  13 255]
  [116 194  13 255]
  [116 194  13 255]]

 [[152 198   9 255]
  [152 198   9 255]
  [152 198   9 255]
  ...
  [116 194  13 255]
  [116 194  

# Add Vehicle

In [5]:
# Add traffic to the simulation
for i in range(settings.number_of_vehicles):
    vehicle_bp = random.choice(bp_lib.filter("vehicle"))
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

    if npc is not None:
        print("Vehicle {} spawned".format(npc.id))

# Set the all vehicles in motion using the Traffic Manager
for v in world.get_actors().filter("*vehicle*"):
    v.set_autopilot(True)

Vehicle 97 spawned
Vehicle 98 spawned
Vehicle 99 spawned
Vehicle 100 spawned
Vehicle 101 spawned
Vehicle 102 spawned
Vehicle 103 spawned


# Add Pedestrians

In [4]:
SpawnActor = carla.command.SpawnActor
walkers_list = []
all_id = []

# 2. we spawn the walker object
batch = []
walker_speed = []
for i in range(settings.number_of_walkers):
    spawn_point = random.choice(spawn_points)
    walker_bp = random.choice(get_actor_blueprints(world, "walker.pedestrian.*", "all"))
    # set as not invincible
    if walker_bp.has_attribute("is_invincible"):
        walker_bp.set_attribute("is_invincible", "false")
    # set the max speed
    if walker_bp.has_attribute("speed"):
        if random.random() > settings.percentage_pedestrians_running:
            # walking
            walker_speed.append(walker_bp.get_attribute("speed").recommended_values[1])
        else:
            # running
            walker_speed.append(walker_bp.get_attribute("speed").recommended_values[2])
    else:
        print("Walker has no speed")
        walker_speed.append(0.0)
    batch.append(SpawnActor(walker_bp, spawn_point))
results = client.apply_batch_sync(batch, True)
walker_speed2 = []
for i in range(len(results)):
    if results[i].error:
        print(results[i].error)
    else:
        walkers_list.append({"id": results[i].actor_id})
        walker_speed2.append(walker_speed[i])
walker_speed = walker_speed2

# 3. we spawn the walker controller
batch = []
walker_controller_bp = world.get_blueprint_library().find("controller.ai.walker")
for i in range(len(walkers_list)):
    batch.append(
        SpawnActor(walker_controller_bp, carla.Transform(), walkers_list[i]["id"])
    )
results = client.apply_batch_sync(batch, True)
for i in range(len(results)):
    if results[i].error:
        print(results[i].error)
    else:
        walkers_list[i]["con"] = results[i].actor_id

# 4. we put together the walkers and controllers id to get the objects from their id
for i in range(len(walkers_list)):
    all_id.append(walkers_list[i]["con"])
    all_id.append(walkers_list[i]["id"])
all_actors = world.get_actors(all_id)
world.tick()

# 5. initialize each controller and set target to walk to (list is [controler, actor, controller, actor ...])
# set how many pedestrians can cross the road
world.set_pedestrians_cross_factor(settings.percentage_pedestrians_crossing)
for i in range(0, len(all_id), 2):
    all_actors[i].start()

    # set walk to random point
    all_actors[i].go_to_location(world.get_random_location_from_navigation())
    # max speed
    all_actors[i].set_max_speed(float(walker_speed[int(i / 2)]))

# Clear Scene

In [44]:
# Stop the camera when we've recorded enough data
[camera.stop() for camera in cameras]
[remove_actors(world, actor) for actor in ["walker", "vehicle"]]

KeyError: 104

[None, None]